In [6]:
# data_generation.py
import pandas as pd
import numpy as np
from faker import Faker
import random

def generate_building_dataset(n_samples=5000):
    """
    Generate synthetic building energy dataset with all 34 features
    """
    np.random.seed(42)
    random.seed(42)
    fake = Faker()
    
    data = {}
    
    # === BASIC BUILDING CHARACTERISTICS ===
    data['total_floor_area'] = np.random.uniform(500, 50000, n_samples)  # m²
    data['number_of_floors'] = np.random.randint(1, 50, n_samples)
    data['occupant_count'] = np.random.randint(10, 2000, n_samples)
    data['building_age'] = np.random.randint(1, 100, n_samples)
    
    # === SYSTEM EFFICIENCIES ===
    data['hvac_efficiency'] = np.random.uniform(0.6, 0.95, n_samples)  # COP ratio (0.6-0.95)
    data['lighting_power_density'] = np.random.uniform(5, 20, n_samples)  # W/m²
    data['equipment_power_density'] = np.random.uniform(3, 15, n_samples)  # W/m²
    
    # === ENVIRONMENTAL FACTORS ===
    # For variety, use different climate distributions
    climate_zones = np.random.choice([1, 2, 3, 4, 5, 6, 7], n_samples, p=[0.1, 0.15, 0.2, 0.25, 0.15, 0.1, 0.05])
    
    # Generate temperatures based on climate zone
    temperatures = []
    for zone in climate_zones:
        if zone in [1, 2]:  # Hot climates
            temperatures.append(np.random.uniform(25, 40))
        elif zone in [3, 4]:  # Temperate
            temperatures.append(np.random.uniform(15, 30))
        else:  # Cold climates
            temperatures.append(np.random.uniform(-5, 20))
    
    data['outdoor_temperature'] = np.array(temperatures)
    data['humidity'] = np.random.uniform(20, 90, n_samples)  # Percentage
    data['climate_zone_numeric'] = climate_zones
    
    # === OPERATIONAL FACTORS ===
    data['occupancy_hours'] = np.random.uniform(8, 24, n_samples)
    data['weekday_numeric'] = np.random.randint(0, 7, n_samples)  # 0=Monday, 6=Sunday
    data['has_elevator_numeric'] = np.random.choice([0, 1], n_samples, p=[0.3, 0.7])
    data['floor_height'] = np.random.uniform(3, 5, n_samples)  # meters
    
    # === BUILDING TYPE (one-hot encoded) ===
    building_types = np.random.choice([0, 1, 2, 3, 4], n_samples, p=[0.4, 0.2, 0.2, 0.1, 0.1])
    data['building_type_numeric'] = building_types
    
    # === BUILDING HEIGHT CATEGORIES ===
    data['is_single_story'] = np.where(data['number_of_floors'] == 1, 1, 0)
    data['is_low_rise'] = np.where((data['number_of_floors'] >= 2) & (data['number_of_floors'] <= 4), 1, 0)
    data['is_mid_rise'] = np.where((data['number_of_floors'] >= 5) & (data['number_of_floors'] <= 9), 1, 0)
    data['is_high_rise'] = np.where(data['number_of_floors'] >= 10, 1, 0)
    
    # === CALCULATE DERIVED FEATURES ===
    data['floor_area_per_floor'] = data['total_floor_area'] / data['number_of_floors']
    data['occupant_density'] = data['occupant_count'] / data['total_floor_area']
    
    # === ENGINEERED FEATURES (based on research insights) ===
    # 1. HVAC floor factor (from sensitivity analysis - 38.18% impact)
    data['hvac_floor_factor'] = data['hvac_efficiency'] * (data['number_of_floors'] / 10)
    
    # 2. Vertical lighting factor (15.07% impact)
    data['vertical_lighting_factor'] = data['lighting_power_density'] * np.sqrt(data['number_of_floors'])
    
    # 3. Vertical occupancy impact (14.48% impact)
    data['vertical_occupancy_impact'] = data['occupant_density'] * (data['number_of_floors'] ** 0.5)
    
    # 4. Elevator energy estimate
    data['elevator_energy_estimate'] = data['has_elevator_numeric'] * data['number_of_floors'] * 100
    
    # 5. Stack effect intensity (buoyancy-driven airflow)
    data['stack_effect_intensity'] = (data['number_of_floors'] * data['floor_height'] * 
                                     np.abs(data['outdoor_temperature'] - 22)) / 100
    
    # 6. Envelope efficiency score (combining age and climate)
    data['envelope_efficiency_score'] = ((100 - data['building_age']) / 100) * (1 - data['climate_zone_numeric'] / 10)
    data['envelope_efficiency_score'] = np.clip(data['envelope_efficiency_score'], 0.1, 1.0)
    
    # 7. Floor compactness (shape efficiency)
    data['floor_compactness'] = data['total_floor_area'] / (data['number_of_floors'] * data['floor_area_per_floor'])
    data['floor_compactness'] = np.clip(data['floor_compactness'], 0.5, 2.0)
    
    # 8. Thermal mass effect (based on construction type and age)
    # Older buildings often have better thermal mass
    data['thermal_mass_effect'] = 0.5 + (data['building_age'] / 200) + np.random.uniform(-0.1, 0.1, n_samples)
    data['thermal_mass_effect'] = np.clip(data['thermal_mass_effect'], 0.3, 2.0)
    
    # 9. Comprehensive efficiency score (weighted sum from research)
    data['comprehensive_efficiency_score'] = (
        data['hvac_efficiency'] * 0.3818 +  # Weight from sensitivity analysis
        data['envelope_efficiency_score'] * 0.3 +
        (1 - data['lighting_power_density'] / 30) * 0.1507
    )
    
    # 10. Degree days calculations
    data['cooling_degree_hours'] = np.maximum(data['outdoor_temperature'] - 18, 0) * 24
    data['heating_degree_hours'] = np.maximum(18 - data['outdoor_temperature'], 0) * 24
    
    # 11. Humidity comfort penalty (optimal humidity ~50%)
    data['humidity_comfort_penalty'] = np.abs(data['humidity'] - 50) / 100
    
    # 12. Usage intensity (occupancy hours normalized by area)
    data['usage_intensity'] = (data['occupant_count'] * data['occupancy_hours']) / data['total_floor_area']
    
    # === CREATE DATAFRAME ===
    df = pd.DataFrame(data)
    
    # === CALCULATE ENERGY CONSUMPTION (TARGET VARIABLE) ===
    # Base formula incorporating key factors from research sensitivity analysis
    base_energy = (
        df['total_floor_area'] * 0.8 +           # Area impact (kWh/m²)
        df['occupant_count'] * 120 +             # Occupant impact (kWh/person)
        df['hvac_efficiency'] * -5000 +          # Efficiency impact (negative: higher efficiency = lower energy)
        df['lighting_power_density'] * 200 +     # Lighting impact
        df['outdoor_temperature'] * 50 +         # Temperature impact
        df['cooling_degree_hours'] * 0.5 +       # Cooling needs
        df['heating_degree_hours'] * 0.3 +       # Heating needs
        df['building_age'] * 10 +                # Age penalty
        df['stack_effect_intensity'] * 100 +     # Stack effect penalty
        df['vertical_lighting_factor'] * 50 +    # Vertical lighting impact
        df['vertical_occupancy_impact'] * 1000 + # Vertical occupancy impact
        df['humidity_comfort_penalty'] * 1000 +  # Humidity penalty
        df['usage_intensity'] * 1000             # Usage intensity impact
    )
    
    # Add noise (realistic variation)
    noise = np.random.normal(0, 0.2, n_samples) * base_energy
    
    # Ensure positive values and add some outliers (5% chance)
    outliers = np.random.choice([0, 1], n_samples, p=[0.95, 0.05])
    outlier_factor = np.where(outliers == 1, np.random.uniform(1.5, 3.0, n_samples), 1.0)
    
    # Final energy consumption calculation
    df['energy_consumption_kwh'] = np.maximum(
        (base_energy + noise) * outlier_factor, 
        1000  # Minimum consumption
    )
    
    # Add building ID and name for realism
    df['building_id'] = [f'B{str(i).zfill(5)}' for i in range(n_samples)]
    df['building_name'] = [f'{fake.company()} Building' for _ in range(n_samples)]
    df['location'] = [fake.city() for _ in range(n_samples)]
    
    # Add some realistic correlations
    # Older buildings tend to have lower HVAC efficiency
    age_efficiency_corr = np.random.choice([0, 1], n_samples, p=[0.3, 0.7])
    df.loc[age_efficiency_corr == 1, 'hvac_efficiency'] *= (1 - df.loc[age_efficiency_corr == 1, 'building_age'] / 200)
    
    # High-rise buildings more likely to have elevators
    high_rise_mask = df['is_high_rise'] == 1
    df.loc[high_rise_mask, 'has_elevator_numeric'] = 1
    
    return df

def save_dataset(df, filename='building_energy_dataset.csv'):
    """Save dataset to CSV"""
    # Reorder columns for better readability
    feature_columns = [
        'building_id', 'building_name', 'location',
        'total_floor_area', 'number_of_floors', 'occupant_count', 
        'building_age', 'floor_area_per_floor', 'occupant_density',
        'hvac_efficiency', 'lighting_power_density', 'equipment_power_density',
        'outdoor_temperature', 'humidity', 'occupancy_hours', 
        'weekday_numeric', 'has_elevator_numeric', 'floor_height',
        'building_type_numeric', 'climate_zone_numeric',
        'is_single_story', 'is_low_rise', 'is_mid_rise', 'is_high_rise',
        'hvac_floor_factor', 'vertical_lighting_factor', 'vertical_occupancy_impact',
        'elevator_energy_estimate', 'stack_effect_intensity', 'envelope_efficiency_score',
        'floor_compactness', 'thermal_mass_effect', 'comprehensive_efficiency_score',
        'cooling_degree_hours', 'heating_degree_hours', 'humidity_comfort_penalty',
        'usage_intensity', 'energy_consumption_kwh'
    ]
    
    df = df[feature_columns]
    df.to_csv(filename, index=False)
    print(f"Dataset saved to {filename}")
    print(f"Shape: {df.shape}")
    print(f"Features: {len(feature_columns) - 1} (plus target variable)")
    return df

# Create and explore dataset
if __name__ == "__main__":
    # Generate dataset
    print("Generating synthetic building energy dataset...")
    df = generate_building_dataset(n_samples=5000)
    
    # Save dataset
    df = save_dataset(df, 'building_energy_dataset.csv')
    
    # Display dataset info
    print("\n=== Dataset Information ===")
    print(f"Total samples: {len(df)}")
    print(f"Features: {len(df.columns) - 1}")  # Excluding target
    
    # Show basic statistics
    print("\n=== Feature Statistics ===")
    stats_df = pd.DataFrame({
        'Feature': df.columns,
        'Type': df.dtypes,
        'Non-Null': df.count(),
        'Null': df.isnull().sum(),
        'Mean': df.mean(numeric_only=True),
        'Std': df.std(numeric_only=True),
        'Min': df.min(numeric_only=True),
        'Max': df.max(numeric_only=True)
    })
    
    # Display for key features
    key_features = [
        'total_floor_area', 'number_of_floors', 'occupant_count',
        'building_age', 'energy_consumption_kwh', 'hvac_efficiency',
        'lighting_power_density', 'outdoor_temperature'
    ]
    
    print("\n=== Key Features Statistics ===")
    for feature in key_features:
        if feature in df.columns:
            print(f"\n{feature}:")
            print(f"  Mean: {df[feature].mean():.2f}")
            print(f"  Std:  {df[feature].std():.2f}")
            print(f"  Min:  {df[feature].min():.2f}")
            print(f"  Max:  {df[feature].max():.2f}")
    
    # Display correlation with target
    print("\n=== Top Correlations with Energy Consumption ===")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    correlations = df[numeric_cols].corr()['energy_consumption_kwh'].abs().sort_values(ascending=False)
    print(correlations.head(15))
    
    # Save a smaller sample for testing
    df_sample = df.head(100)
    df_sample.to_csv('building_energy_sample.csv', index=False)
    print("\nSample dataset saved to 'building_energy_sample.csv'")
    
    # Create a metadata file - FIXED VERSION
    metadata = {
        'feature': [
            'total_floor_area', 'number_of_floors', 'occupant_count', 
            'building_age', 'floor_area_per_floor', 'occupant_density',
            'hvac_efficiency', 'lighting_power_density', 'equipment_power_density',
            'outdoor_temperature', 'humidity', 'occupancy_hours',
            'weekday_numeric', 'has_elevator_numeric', 'floor_height',
            'building_type_numeric', 'climate_zone_numeric', 'is_single_story',
            'is_low_rise', 'is_mid_rise', 'is_high_rise', 'hvac_floor_factor',
            'vertical_lighting_factor', 'vertical_occupancy_impact', 'elevator_energy_estimate',
            'stack_effect_intensity', 'envelope_efficiency_score', 'floor_compactness',
            'thermal_mass_effect', 'comprehensive_efficiency_score', 'cooling_degree_hours',
            'heating_degree_hours', 'humidity_comfort_penalty', 'usage_intensity',
            'energy_consumption_kwh'  # Added target variable
        ],
        'description': [
            'Total floor area in square meters',
            'Number of floors in the building',
            'Number of regular occupants',
            'Age of building in years',
            'Average area per floor (m²/floor)',
            'Occupants per square meter',
            'HVAC system efficiency ratio (0-1, higher is better)',
            'Lighting power density in W/m²',
            'Equipment power density in W/m²',
            'Average outdoor temperature in °C',
            'Average relative humidity in %',
            'Average daily occupancy hours',
            'Weekday numeric (0=Monday, 6=Sunday)',
            'Binary indicator for elevator presence',
            'Average floor height in meters',
            'Building type encoded as integer',
            'Climate zone code (1-7)',
            'Binary: 1 if single story building',
            'Binary: 1 if low-rise (2-4 floors)',
            'Binary: 1 if mid-rise (5-9 floors)',
            'Binary: 1 if high-rise (10+ floors)',
            'HVAC efficiency scaled by floor count',
            'Lighting impact considering vertical distribution',
            'Occupancy impact considering vertical distribution',
            'Estimated elevator energy consumption',
            'Stack effect intensity from temperature difference',
            'Building envelope thermal performance score',
            'Floor plan compactness metric',
            'Thermal mass effect on energy performance',
            'Weighted comprehensive efficiency score',
            'Cooling degree hours based on temperature',
            'Heating degree hours based on temperature',
            'Penalty for humidity deviation from comfort zone',
            'Intensity of building usage',
            'Annual energy consumption (target variable)'
        ],
        'unit': [
            'm²', 'count', 'count', 'years', 'm²/floor', 'person/m²',
            'ratio', 'W/m²', 'W/m²', '°C', '%', 'hours',
            '0-6', '0/1', 'm', '0-4', '1-7',
            '0/1', '0/1', '0/1', '0/1', 'dimensionless',
            'dimensionless', 'dimensionless', 'kWh',
            'dimensionless', '0-1', 'ratio',
            'dimensionless', '0-1', '°C-hours',
            '°C-hours', 'ratio', 'dimensionless',
            'kWh'
        ]
    }
    
    metadata_df = pd.DataFrame(metadata)
    metadata_df.to_csv('dataset_metadata.csv', index=False)
    print("Metadata saved to 'dataset_metadata.csv'")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/opt/anaconda3/lib/python3.12/site-

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/opt/anaconda3/lib/python3.12/site-

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/opt/anaconda3/lib/python3.12/site-

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/opt/anaconda3/lib/python3.12/site-

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



Generating synthetic building energy dataset...
Dataset saved to building_energy_dataset.csv
Shape: (5000, 38)
Features: 37 (plus target variable)

=== Dataset Information ===
Total samples: 5000
Features: 37

=== Feature Statistics ===

=== Key Features Statistics ===

total_floor_area:
  Mean: 25093.18
  Std:  14336.87
  Min:  500.58
  Max:  49986.02

number_of_floors:
  Mean: 24.65
  Std:  14.05
  Min:  1.00
  Max:  49.00

occupant_count:
  Mean: 1019.94
  Std:  568.75
  Min:  10.00
  Max:  1998.00

building_age:
  Mean: 49.34
  Std:  28.61
  Min:  1.00
  Max:  99.00

energy_consumption_kwh:
  Mean: 158430.44
  Std:  97824.62
  Min:  5703.35
  Max:  913660.81

hvac_efficiency:
  Mean: 0.65
  Std:  0.15
  Min:  0.30
  Max:  0.95

lighting_power_density:
  Mean: 12.66
  Std:  4.34
  Min:  5.00
  Max:  20.00

outdoor_temperature:
  Mean: 20.60
  Std:  10.76
  Min:  -5.00
  Max:  39.99

=== Top Correlations with Energy Consumption ===
energy_consumption_kwh       1.000000
occupant_count

In [1]:
# complete_ml_pipeline.py
print("=" * 70)
print("🏢 COMPLETE BUILDING ENERGY ML PIPELINE")
print("=" * 70)

# Import all packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb
import joblib
import warnings
warnings.filterwarnings('ignore')

print("✅ All imports successful!")

# Load your dataset
print("\n📊 Loading dataset...")
try:
    df = pd.read_csv('building_energy_dataset.csv')
    print(f"   Dataset loaded: {len(df)} rows, {len(df.columns)} columns")
except FileNotFoundError:
    print("❌ Dataset not found. Creating sample data...")
    # Create sample dataset
    np.random.seed(42)
    n_samples = 1000
    df = pd.DataFrame({
        'total_floor_area': np.random.uniform(500, 50000, n_samples),
        'number_of_floors': np.random.randint(1, 20, n_samples),
        'occupant_count': np.random.randint(10, 1000, n_samples),
        'building_age': np.random.randint(1, 50, n_samples),
        'hvac_efficiency': np.random.uniform(0.6, 0.95, n_samples),
        'lighting_power_density': np.random.uniform(5, 20, n_samples),
        'equipment_power_density': np.random.uniform(3, 15, n_samples),
        'outdoor_temperature': np.random.uniform(10, 35, n_samples),
        'humidity': np.random.uniform(30, 80, n_samples),
        'occupancy_hours': np.random.uniform(8, 16, n_samples)
    })
    
    # Calculate energy consumption
    df['energy_consumption_kwh'] = (
        df['total_floor_area'] * 0.8 +
        df['occupant_count'] * 120 +
        df['hvac_efficiency'] * -5000 +
        df['lighting_power_density'] * 200 +
        df['outdoor_temperature'] * 50 +
        df['building_age'] * 10
    ) * np.random.uniform(0.8, 1.2, n_samples)
    
    df.to_csv('building_energy_dataset.csv', index=False)
    print(f"   Sample dataset created and saved")

# Display basic info
print(f"\n📈 Dataset Information:")
print(f"   Target variable: energy_consumption_kwh")
print(f"   Range: {df['energy_consumption_kwh'].min():,.0f} to {df['energy_consumption_kwh'].max():,.0f} kWh")
print(f"   Mean: {df['energy_consumption_kwh'].mean():,.0f} kWh")

# Prepare features
feature_list = [
    'total_floor_area', 'number_of_floors', 'occupant_count',
    'building_age', 'hvac_efficiency', 'lighting_power_density',
    'equipment_power_density', 'outdoor_temperature', 'humidity',
    'occupancy_hours'
]

# Use available features
available_features = [f for f in feature_list if f in df.columns]
X = df[available_features]
y = df['energy_consumption_kwh']

print(f"\n🔧 Using {len(available_features)} features:")
for i, feature in enumerate(available_features, 1):
    print(f"   {i:2d}. {feature}")

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\n📊 Data Split:")
print(f"   Training samples: {len(X_train)}")
print(f"   Testing samples: {len(X_test)}")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n" + "=" * 70)
print("🤖 TRAINING MACHINE LEARNING MODELS")
print("=" * 70)

# Define models
models = {
    'Random Forest': RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        random_state=42
    ),
    'XGBoost': xgb.XGBRegressor(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        random_state=42,
        verbosity=0
    ),
    'LightGBM': lgb.LGBMRegressor(
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        random_state=42,
        verbose=-1
    ),
    'Gradient Boosting': GradientBoostingRegressor(
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        random_state=42
    )
}

# Train and evaluate
results = {}
for name, model in models.items():
    print(f"\n   Training {name}...")
    model.fit(X_train_scaled, y_train)
    
    # Predict
    y_pred = model.predict(X_test_scaled)
    
    # Calculate metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    
    results[name] = {
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'MAPE': mape,
        'model': model
    }
    
    print(f"     MAE: {mae:,.0f} kWh")
    print(f"     RMSE: {rmse:,.0f} kWh")
    print(f"     R²: {r2:.4f}")
    print(f"     MAPE: {mape:.2f}%")

# Find best model
best_model_name = max(results, key=lambda x: results[x]['R2'])
best_model = results[best_model_name]['model']

print(f"\n🏆 Best Model: {best_model_name}")
print(f"   R² Score: {results[best_model_name]['R2']:.4f}")

print("\n" + "=" * 70)
print("📊 MODEL COMPARISON")
print("=" * 70)

# Display comparison
comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'MAE (kWh)': [f"{results[m]['MAE']:,.0f}" for m in results.keys()],
    'RMSE (kWh)': [f"{results[m]['RMSE']:,.0f}" for m in results.keys()],
    'R² Score': [f"{results[m]['R2']:.4f}" for m in results.keys()],
    'MAPE (%)': [f"{results[m]['MAPE']:.2f}" for m in results.keys()]
}).sort_values('R² Score', ascending=False)

print(comparison_df.to_string(index=False))

# Feature importance from best model
print(f"\n🔝 Feature Importance ({best_model_name}):")
if hasattr(best_model, 'feature_importances_'):
    importances = pd.DataFrame({
        'Feature': available_features,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print(importances.to_string(index=False))
else:
    print("   (Feature importance not available for this model type)")

print("\n" + "=" * 70)
print("💾 SAVING MODEL AND ARTIFACTS")
print("=" * 70)

# Save everything
artifacts = {
    'model': best_model,
    'scaler': scaler,
    'features': available_features,
    'results': results,
    'best_model_name': best_model_name
}

joblib.dump(artifacts, 'building_energy_artifacts.joblib')
print("✅ Model artifacts saved to 'building_energy_artifacts.joblib'")

# Save just the model for production
joblib.dump(best_model, 'building_energy_model.joblib')
print("✅ Production model saved to 'building_energy_model.joblib'")

# Example prediction
print(f"\n🔮 EXAMPLE PREDICTION")
print("=" * 70)

# Create a sample building
sample_building = {
    'total_floor_area': 10000,
    'number_of_floors': 5,
    'occupant_count': 200,
    'building_age': 10,
    'hvac_efficiency': 0.85,
    'lighting_power_density': 12,
    'equipment_power_density': 8,
    'outdoor_temperature': 25,
    'humidity': 60,
    'occupancy_hours': 10
}

# Prepare sample
sample_df = pd.DataFrame([sample_building])
for feature in available_features:
    if feature not in sample_df.columns:
        sample_df[feature] = 0  # Fill missing with 0

sample_df = sample_df[available_features]
sample_scaled = scaler.transform(sample_df)

# Predict
prediction = best_model.predict(sample_scaled)[0]

print(f"\n📐 Sample Building Characteristics:")
for key, value in sample_building.items():
    if key in available_features:
        print(f"   {key:25s}: {value}")

print(f"\n🔮 Predicted Annual Energy Consumption:")
print(f"   {prediction:,.0f} kWh")

# Breakdown
print(f"\n📊 Energy Breakdown Estimate:")
print(f"   Per m²: {prediction/sample_building['total_floor_area']:.1f} kWh/m²")
print(f"   Per occupant: {prediction/sample_building['occupant_count']:.0f} kWh/person")
print(f"   Monthly average: {prediction/12:,.0f} kWh/month")

# CO2 emissions estimate
co2_kg = prediction * 0.85  # kg CO2 per kWh
print(f"   Estimated CO₂ emissions: {co2_kg/1000:,.1f} tons CO₂/year")

print("\n" + "=" * 70)
print("🎉 ML PIPELINE COMPLETED SUCCESSFULLY!")
print("=" * 70)
print("\nNext steps:")
print("1. Run: streamlit run streamlit_app.py (for web interface)")
print("2. Use the saved model for predictions")
print("3. Analyze feature importance for energy efficiency improvements")

🏢 COMPLETE BUILDING ENERGY ML PIPELINE
✅ All imports successful!

📊 Loading dataset...
   Dataset loaded: 5000 rows, 38 columns

📈 Dataset Information:
   Target variable: energy_consumption_kwh
   Range: 5,703 to 913,661 kWh
   Mean: 158,430 kWh

🔧 Using 10 features:
    1. total_floor_area
    2. number_of_floors
    3. occupant_count
    4. building_age
    5. hvac_efficiency
    6. lighting_power_density
    7. equipment_power_density
    8. outdoor_temperature
    9. humidity
   10. occupancy_hours

📊 Data Split:
   Training samples: 4000
   Testing samples: 1000

🤖 TRAINING MACHINE LEARNING MODELS

   Training Random Forest...
     MAE: 37,071 kWh
     RMSE: 69,266 kWh
     R²: 0.5346
     MAPE: 23.31%

   Training XGBoost...
     MAE: 38,236 kWh
     RMSE: 71,999 kWh
     R²: 0.4972
     MAPE: 24.35%

   Training LightGBM...
     MAE: 37,750 kWh
     RMSE: 70,045 kWh
     R²: 0.5241
     MAPE: 24.47%

   Training Gradient Boosting...
     MAE: 37,400 kWh
     RMSE: 70,192 kWh
  

In [3]:
# train_energy_model.py
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

print("🔧 TRAINING BUILDING ENERGY PREDICTION MODEL")
print("=" * 60)

# Load or create dataset
def load_dataset():
    try:
        df = pd.read_csv('building_energy_dataset.csv')
        print(f"📊 Dataset loaded: {len(df)} samples")
        return df
    except FileNotFoundError:
        print("⚠️  No dataset found. Creating sample data...")
        return create_sample_dataset()

def create_sample_dataset():
    np.random.seed(42)
    n_samples = 1000
    
    # Create realistic building data
    df = pd.DataFrame({
        'total_floor_area': np.random.uniform(500, 50000, n_samples),
        'number_of_floors': np.random.randint(1, 20, n_samples),
        'occupant_count': np.random.randint(10, 1000, n_samples),
        'building_age': np.random.randint(1, 50, n_samples),
        'hvac_efficiency': np.random.uniform(0.6, 0.95, n_samples),
        'lighting_power_density': np.random.uniform(5, 20, n_samples),
        'equipment_power_density': np.random.uniform(3, 15, n_samples),
        'outdoor_temperature': np.random.uniform(10, 35, n_samples),
        'humidity': np.random.uniform(30, 80, n_samples),
        'occupancy_hours': np.random.uniform(8, 16, n_samples),
        'has_elevator': np.random.choice([0, 1], n_samples),
        'building_type_office': np.random.choice([0, 1], n_samples),
        'building_type_residential': np.random.choice([0, 1], n_samples),
        'building_type_commercial': np.random.choice([0, 1], n_samples)
    })
    
    # Ensure only one building type is active
    building_type_cols = ['building_type_office', 'building_type_residential', 'building_type_commercial']
    for i in range(len(df)):
        if df.loc[i, building_type_cols].sum() == 0:
            df.loc[i, np.random.choice(building_type_cols)] = 1
    
    # Calculate realistic energy consumption
    base_energy = (
        df['total_floor_area'] * 0.5 +  # Base load per m²
        df['occupant_count'] * 100 +     # Occupant load
        df['outdoor_temperature'] * 30 +  # Climate impact
        (50 - df['hvac_efficiency'] * 100) * 50 +  # Efficiency impact
        df['building_age'] * 5           # Age impact
    )
    
    # Add building type adjustments
    df['energy_consumption_kwh'] = base_energy * np.random.uniform(0.9, 1.1, n_samples)
    
    # Save dataset
    df.to_csv('building_energy_dataset.csv', index=False)
    print(f"✅ Sample dataset created with {n_samples} samples")
    return df

# Prepare data for training
def prepare_data(df):
    # Define features (remove target variable)
    feature_columns = [
        'total_floor_area', 'number_of_floors', 'occupant_count',
        'building_age', 'hvac_efficiency', 'lighting_power_density',
        'equipment_power_density', 'outdoor_temperature', 'humidity',
        'occupancy_hours', 'has_elevator',
        'building_type_office', 'building_type_residential', 'building_type_commercial'
    ]
    
    # Use only columns that exist in dataframe
    available_features = [f for f in feature_columns if f in df.columns]
    
    print(f"\n🎯 Target variable: energy_consumption_kwh")
    print(f"🔧 Using {len(available_features)} features")
    
    X = df[available_features]
    y = df['energy_consumption_kwh']
    
    return X, y, available_features

# Train the model
def train_model(X, y, features):
    print("\n📊 Splitting data (80% train, 20% test)...")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    print(f"   Training samples: {len(X_train)}")
    print(f"   Testing samples: {len(X_test)}")
    
    # Scale features
    print("\n🔧 Scaling features...")
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Choose and train model (XGBoost usually performs best)
    print("\n🤖 Training XGBoost model...")
    model = xgb.XGBRegressor(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        verbosity=0
    )
    
    model.fit(X_train_scaled, y_train)
    
    # Quick evaluation
    from sklearn.metrics import r2_score, mean_absolute_error
    y_pred = model.predict(X_test_scaled)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    print(f"\n📈 Model Performance:")
    print(f"   R² Score: {r2:.4f}")
    print(f"   MAE: {mae:,.0f} kWh")
    print(f"   Avg Error: {mae/y_test.mean()*100:.1f}% of mean energy use")
    
    return model, scaler, features, r2

# Save model artifacts
def save_artifacts(model, scaler, features, r2_score):
    artifacts = {
        'model': model,
        'scaler': scaler,
        'features': features,
        'training_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
        'r2_score': r2_score,
        'model_type': 'XGBoost'
    }
    
    joblib.dump(artifacts, 'building_energy_model.joblib')
    print(f"\n💾 Model saved to 'building_energy_model.joblib'")
    
    # Also save just the model for production
    joblib.dump(model, 'energy_model_xgboost.joblib')
    print("💾 Production model saved to 'energy_model_xgboost.joblib'")
    
    # Save feature list
    with open('model_features.txt', 'w') as f:
        for feature in features:
            f.write(f"{feature}\n")
    print("📝 Feature list saved to 'model_features.txt'")

# Test with sample prediction
def test_sample_prediction(model, scaler, features):
    print("\n🔮 Testing with sample building...")
    
    # Sample building matching your UI
    sample_building = {
        'total_floor_area': 5000,
        'number_of_floors': 5,
        'occupant_count': 200,
        'building_age': 10,
        'hvac_efficiency': 0.85,
        'lighting_power_density': 12,
        'equipment_power_density': 8,
        'outdoor_temperature': 25,
        'humidity': 60,
        'occupancy_hours': 10,
        'has_elevator': 1,
        'building_type_office': 1,
        'building_type_residential': 0,
        'building_type_commercial': 0
    }
    
    # Create dataframe with correct feature order
    sample_df = pd.DataFrame([sample_building])
    
    # Ensure all features are present
    for feature in features:
        if feature not in sample_df.columns:
            sample_df[feature] = 0  # Default value for missing
    
    sample_df = sample_df[features]
    sample_scaled = scaler.transform(sample_df)
    
    prediction = model.predict(sample_scaled)[0]
    
    print(f"\n📐 Sample Building:")
    print(f"   Area: {sample_building['total_floor_area']} m²")
    print(f"   Occupants: {sample_building['occupant_count']}")
    print(f"   Floors: {sample_building['number_of_floors']}")
    
    print(f"\n🔮 Predicted Annual Energy:")
    print(f"   {prediction:,.0f} kWh")
    print(f"   {prediction/12:,.0f} kWh/month")
    print(f"   {prediction/sample_building['total_floor_area']:.1f} kWh/m²")

# Main training function
def main():
    print("🚀 Starting model training...")
    
    # 1. Load data
    df = load_dataset()
    
    # 2. Prepare data
    X, y, features = prepare_data(df)
    
    # 3. Train model
    model, scaler, features, r2 = train_model(X, y, features)
    
    # 4. Save artifacts
    save_artifacts(model, scaler, features, r2)
    
    # 5. Test prediction
    test_sample_prediction(model, scaler, features)
    
    print("\n" + "=" * 60)
    print("✅ MODEL TRAINING COMPLETED!")
    print("=" * 60)
    print("\n📋 Next steps:")
    print("1. Copy the saved model files to your Streamlit app directory")
    print("2. Update your Streamlit app to load 'building_energy_model.joblib'")
    print("3. Run: streamlit run streamlit_app.py")
    print("\n⚠️  Make sure your Streamlit app expects these features:")
    for i, feature in enumerate(features, 1):
        print(f"   {i:2d}. {feature}")

if __name__ == "__main__":
    main()

🔧 TRAINING BUILDING ENERGY PREDICTION MODEL
🚀 Starting model training...
📊 Dataset loaded: 5000 samples

🎯 Target variable: energy_consumption_kwh
🔧 Using 10 features

📊 Splitting data (80% train, 20% test)...
   Training samples: 4000
   Testing samples: 1000

🔧 Scaling features...

🤖 Training XGBoost model...

📈 Model Performance:
   R² Score: 0.4850
   MAE: 39,364 kWh
   Avg Error: 24.7% of mean energy use

💾 Model saved to 'building_energy_model.joblib'
💾 Production model saved to 'energy_model_xgboost.joblib'
📝 Feature list saved to 'model_features.txt'

🔮 Testing with sample building...

📐 Sample Building:
   Area: 5000 m²
   Occupants: 200
   Floors: 5

🔮 Predicted Annual Energy:
   75,569 kWh
   6,297 kWh/month
   15.1 kWh/m²

✅ MODEL TRAINING COMPLETED!

📋 Next steps:
1. Copy the saved model files to your Streamlit app directory
2. Update your Streamlit app to load 'building_energy_model.joblib'
3. Run: streamlit run streamlit_app.py

⚠️  Make sure your Streamlit app expects t

In [2]:
# streamlit_app.py
import streamlit as st
import pandas as pd
import numpy as np
import joblib
import plotly.graph_objects as go
import plotly.express as px

# Page configuration
st.set_page_config(
    page_title="Building Energy Consumption Predictor",
    page_icon="🏢",
    layout="wide"
)

# Title
st.title("🏢 Building Energy Consumption Predictor")
st.markdown("""
Predict annual energy consumption based on building characteristics and operational parameters.
Based on research: *"Enhancing the Accuracy of Building Performance Simulation through Post-Occupancy Calibration"*
""")

# Sidebar for navigation
st.sidebar.title("Navigation")
app_mode = st.sidebar.selectbox(
    "Choose a mode",
    ["📊 Predict Energy", "📈 Model Insights", "📚 About", "📁 Upload Data"]
)

# Load model
@st.cache_resource
def load_model():
    try:
        artifacts = joblib.load('building_energy_artifacts.joblib')
        return artifacts
    except:
        st.error("Model not found. Please train the model first.")
        return None

# Load dataset info
@st.cache_data
def load_dataset_info():
    try:
        df = pd.read_csv('building_energy_dataset.csv')
        return df
    except:
        return None

if app_mode == "📊 Predict Energy":
    st.header("Building Energy Consumption Prediction")
    
    col1, col2, col3 = st.columns(3)
    
    with col1:
        st.subheader("Basic Information")
        total_floor_area = st.number_input(
            "Total Floor Area (m²)",
            min_value=100,
            max_value=100000,
            value=5000,
            step=100,
            help="Total floor area of the building"
        )
        
        number_of_floors = st.number_input(
            "Number of Floors",
            min_value=1,
            max_value=100,
            value=5,
            step=1
        )
        
        building_age = st.number_input(
            "Building Age (years)",
            min_value=0,
            max_value=200,
            value=10,
            step=1
        )
        
        building_type = st.selectbox(
            "Building Type",
            ["Office", "Residential", "Commercial", "Educational", "Healthcare", "Mixed-Use"],
            index=0
        )
    
    with col2:
        st.subheader("Occupancy & Usage")
        occupant_count = st.number_input(
            "Number of Occupants",
            min_value=1,
            max_value=5000,
            value=200,
            step=10
        )
        
        occupancy_hours = st.slider(
            "Daily Operating Hours",
            min_value=0,
            max_value=24,
            value=10,
            step=1,
            help="Average hours the building is occupied per day"
        )
        
        has_elevator = st.radio(
            "Has Elevator?",
            ["Yes", "No"],
            horizontal=True
        )
        
        equipment_load = st.slider(
            "Equipment Load Factor",
            min_value=0.5,
            max_value=2.0,
            value=1.0,
            step=0.1,
            help="Relative equipment usage intensity"
        )
    
    with col3:
        st.subheader("Systems & Environment")
        hvac_efficiency = st.slider(
            "HVAC System Efficiency",
            min_value=0.5,
            max_value=1.0,
            value=0.85,
            step=0.01,
            help="Higher values indicate more efficient systems (COP ratio)"
        )
        
        lighting_power_density = st.slider(
            "Lighting Power Density (W/m²)",
            min_value=5,
            max_value=30,
            value=12,
            step=1
        )
        
        equipment_power_density = st.slider(
            "Equipment Power Density (W/m²)",
            min_value=2,
            max_value=20,
            value=8,
            step=1
        )
        
        outdoor_temperature = st.number_input(
            "Average Outdoor Temperature (°C)",
            min_value=-30,
            max_value=50,
            value=25,
            step=1
        )
        
        humidity = st.slider(
            "Average Relative Humidity (%)",
            min_value=10,
            max_value=100,
            value=60,
            step=5
        )
    
    # Advanced parameters
    with st.expander("⚙️ Advanced Parameters & Research-Based Features"):
        col4, col5 = st.columns(2)
        
        with col4:
            climate_zone = st.selectbox(
                "Climate Zone",
                ["Hot-Humid", "Hot-Dry", "Temperate", "Cold", "Very Cold"],
                index=2
            )
            
            envelope_efficiency = st.slider(
                "Building Envelope Efficiency",
                min_value=0.0,
                max_value=1.0,
                value=0.7,
                step=0.05,
                help="Thermal performance of building envelope"
            )
            
            thermal_mass = st.slider(
                "Thermal Mass Effect",
                min_value=0.5,
                max_value=2.0,
                value=1.2,
                step=0.1,
                help="Effect of building thermal mass on energy consumption"
            )
        
        with col5:
            usage_intensity = st.slider(
                "Usage Intensity Factor",
                min_value=0.5,
                max_value=2.0,
                value=1.0,
                step=0.1,
                help="Overall intensity of building usage"
            )
            
            maintenance_quality = st.slider(
                "Maintenance Quality",
                min_value=0.0,
                max_value=1.0,
                value=0.8,
                step=0.05,
                help="Quality of building systems maintenance"
            )
            
            # Research-based feature from paper
            performance_gap_factor = st.slider(
                "Performance Gap Factor",
                min_value=0.8,
                max_value=1.2,
                value=1.0,
                step=0.01,
                help="Accounts for discrepancy between design and actual performance (from research paper)"
            )
    
    # Prediction button
    if st.button("🔮 Predict Energy Consumption", type="primary", use_container_width=True):
        artifacts = load_model()
        
        if artifacts:
            try:
                # Prepare input features based on trained model
                model = artifacts['model']
                scaler = artifacts['scaler']
                features = artifacts['features']
                
                # Calculate derived values
                occupant_density = occupant_count / total_floor_area if total_floor_area > 0 else 0
                
                # Create input dictionary
                input_data = {}
                for feature in features:
                    if feature == 'total_floor_area':
                        input_data[feature] = total_floor_area
                    elif feature == 'number_of_floors':
                        input_data[feature] = number_of_floors
                    elif feature == 'occupant_count':
                        input_data[feature] = occupant_count
                    elif feature == 'building_age':
                        input_data[feature] = building_age
                    elif feature == 'hvac_efficiency':
                        input_data[feature] = hvac_efficiency
                    elif feature == 'lighting_power_density':
                        input_data[feature] = lighting_power_density
                    elif feature == 'equipment_power_density':
                        input_data[feature] = equipment_power_density * equipment_load
                    elif feature == 'outdoor_temperature':
                        input_data[feature] = outdoor_temperature
                    elif feature == 'humidity':
                        input_data[feature] = humidity
                    elif feature == 'occupancy_hours':
                        input_data[feature] = occupancy_hours
                    else:
                        # Set default value for other features
                        input_data[feature] = 0
                
                # Create DataFrame
                input_df = pd.DataFrame([input_data])
                
                # Ensure correct order
                input_df = input_df[features]
                
                # Scale features
                input_scaled = scaler.transform(input_df)
                
                # Predict
                prediction = model.predict(input_scaled)[0]
                
                # Apply research-based adjustment
                adjusted_prediction = prediction * performance_gap_factor
                
                # Display results
                st.success(f"### Predicted Annual Energy Consumption: **{adjusted_prediction:,.0f} kWh**")
                
                # Create metrics columns
                col1, col2, col3, col4 = st.columns(4)
                with col1:
                    st.metric(
                        "Energy Intensity",
                        f"{adjusted_prediction/total_floor_area:.1f} kWh/m²",
                        delta=f"±{(adjusted_prediction*0.1)/total_floor_area:.1f} kWh/m²"
                    )
                
                with col2:
                    st.metric(
                        "Per Occupant",
                        f"{adjusted_prediction/occupant_count:.0f} kWh/person",
                        delta=f"±{(adjusted_prediction*0.1)/occupant_count:.0f} kWh/person"
                    )
                
                with col3:
                    st.metric(
                        "Monthly Average",
                        f"{adjusted_prediction/12:,.0f} kWh/month",
                        delta=f"±{adjusted_prediction*0.1/12:,.0f} kWh/month"
                    )
                
                with col4:
                    co2_emissions = adjusted_prediction * 0.85  # kg CO2 per kWh
                    st.metric(
                        "CO₂ Emissions",
                        f"{co2_emissions/1000:,.1f} tons",
                        help="Based on average grid emission factor"
                    )
                
                # Visualization
                st.subheader("📊 Energy Consumption Analysis")
                
                tab1, tab2, tab3 = st.tabs(["Breakdown", "Comparison", "Efficiency"])
                
                with tab1:
                    # Energy breakdown
                    breakdown = {
                        'HVAC (38%)': adjusted_prediction * 0.38,
                        'Lighting (15%)': adjusted_prediction * 0.15,
                        'Equipment (15%)': adjusted_prediction * 0.15,
                        'Hot Water (8%)': adjusted_prediction * 0.08,
                        'Other (24%)': adjusted_prediction * 0.24
                    }
                    
                    fig1 = go.Figure(data=[go.Pie(
                        labels=list(breakdown.keys()),
                        values=list(breakdown.values()),
                        hole=0.4,
                        marker_colors=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFEAA7', '#A8E6CF']
                    )])
                    
                    fig1.update_layout(
                        title="Estimated Energy Consumption Breakdown",
                        height=400
                    )
                    
                    st.plotly_chart(fig1, use_container_width=True)
                
                with tab2:
                    # Comparison with similar buildings
                    df = load_dataset_info()
                    if df is not None:
                        similar_buildings = df[
                            (df['total_floor_area'] >= total_floor_area * 0.8) &
                            (df['total_floor_area'] <= total_floor_area * 1.2)
                        ].copy()
                        
                        if len(similar_buildings) > 0:
                            similar_buildings['Type'] = 'Similar Buildings'
                            current_building = pd.DataFrame([{
                                'energy_consumption_kwh': adjusted_prediction,
                                'Type': 'Your Building'
                            }])
                            
                            comparison_df = pd.concat([
                                similar_buildings[['energy_consumption_kwh', 'Type']],
                                current_building
                            ])
                            
                            fig2 = px.box(
                                comparison_df,
                                x='Type',
                                y='energy_consumption_kwh',
                                points='all',
                                title="Comparison with Similar Buildings"
                            )
                            
                            fig2.update_layout(height=400)
                            st.plotly_chart(fig2, use_container_width=True)
                        else:
                            st.info("No similar buildings found in dataset for comparison.")
                
                with tab3:
                    # Efficiency recommendations
                    st.subheader("💡 Energy Efficiency Recommendations")
                    
                    recommendations = []
                    
                    if hvac_efficiency < 0.8:
                        recommendations.append((
                            "Upgrade HVAC System",
                            f"Current efficiency: {hvac_efficiency:.2f}. Target: >0.85",
                            "Potential savings: 15-25%"
                        ))
                    
                    if lighting_power_density > 15:
                        recommendations.append((
                            "Improve Lighting Efficiency",
                            f"Current: {lighting_power_density} W/m². Target: <12 W/m²",
                            "Potential savings: 20-40%"
                        ))
                    
                    if building_age > 20:
                        recommendations.append((
                            "Building Envelope Retrofit",
                            f"Building age: {building_age} years",
                            "Potential savings: 10-30%"
                        ))
                    
                    if not recommendations:
                        st.success("Your building parameters are already efficient!")
                    else:
                        for title, current, savings in recommendations:
                            with st.expander(f"🔧 {title}"):
                                st.write(f"**Current**: {current}")
                                st.write(f"**{savings}**")
                                st.write(f"**Actions**: Regular maintenance, system upgrades, smart controls")
                
                # Download prediction
                prediction_data = {
                    'Parameter': list(input_data.keys()),
                    'Value': list(input_data.values()),
                    'Predicted_Energy_kWh': [adjusted_prediction] * len(input_data)
                }
                
                prediction_df = pd.DataFrame(prediction_data)
                csv = prediction_df.to_csv(index=False)
                
                st.download_button(
                    label="📥 Download Prediction Details",
                    data=csv,
                    file_name="building_energy_prediction.csv",
                    mime="text/csv"
                )
                
            except Exception as e:
                st.error(f"Prediction error: {str(e)}")
                st.info("Please check that all required features are provided.")

elif app_mode == "📈 Model Insights":
    st.header("Model Performance Insights")
    
    artifacts = load_model()
    
    if artifacts:
        st.info(f"**Model Type**: {artifacts.get('best_model_name', 'Unknown')}")
        
        # Display metrics
        st.subheader("Model Performance Metrics")
        
        if 'results' in artifacts:
            results = artifacts['results']
            best_model = artifacts['best_model_name']
            
            col1, col2, col3, col4 = st.columns(4)
            with col1:
                st.metric("R² Score", f"{results[best_model]['R2']:.4f}")
            with col2:
                st.metric("MAE", f"{results[best_model]['MAE']:,.0f} kWh")
            with col3:
                st.metric("RMSE", f"{results[best_model]['RMSE']:,.0f} kWh")
            with col4:
                st.metric("MAPE", f"{results[best_model]['MAPE']:.2f}%")
        
        # Feature importance
        st.subheader("Feature Importance Analysis")
        
        # Load dataset for correlation
        df = load_dataset_info()
        if df is not None and 'features' in artifacts:
            features = artifacts['features']
            
            # Calculate correlations
            corr_data = []
            for feature in features:
                if feature in df.columns:
                    correlation = df[feature].corr(df['energy_consumption_kwh'])
                    corr_data.append({
                        'Feature': feature,
                        'Correlation': abs(correlation),
                        'Direction': 'Positive' if correlation > 0 else 'Negative'
                    })
            
            if corr_data:
                corr_df = pd.DataFrame(corr_data).sort_values('Correlation', ascending=False)
                
                fig = go.Figure()
                
                # Positive correlations
                pos_df = corr_df[corr_df['Direction'] == 'Positive']
                if len(pos_df) > 0:
                    fig.add_trace(go.Bar(
                        x=pos_df['Correlation'],
                        y=pos_df['Feature'],
                        name='Positive Correlation',
                        orientation='h',
                        marker_color='#4ECDC4'
                    ))
                
                # Negative correlations
                neg_df = corr_df[corr_df['Direction'] == 'Negative']
                if len(neg_df) > 0:
                    fig.add_trace(go.Bar(
                        x=neg_df['Correlation'],
                        y=neg_df['Feature'],
                        name='Negative Correlation',
                        orientation='h',
                        marker_color='#FF6B6B'
                    ))
                
                fig.update_layout(
                    title="Feature Correlation with Energy Consumption",
                    xaxis_title="Absolute Correlation Coefficient",
                    height=500,
                    barmode='overlay'
                )
                
                st.plotly_chart(fig, use_container_width=True)
        
        # Model comparison
        if 'results' in artifacts:
            st.subheader("Model Comparison")
            
            models_data = []
            for model_name, metrics in artifacts['results'].items():
                models_data.append({
                    'Model': model_name,
                    'R²': metrics['R2'],
                    'MAE': metrics['MAE'],
                    'RMSE': metrics['RMSE']
                })
            
            models_df = pd.DataFrame(models_data).sort_values('R²', ascending=False)
            st.dataframe(models_df.style.format({
                'R²': '{:.4f}',
                'MAE': '{:,.0f}',
                'RMSE': '{:,.0f}'
            }), use_container_width=True)

elif app_mode == "📚 About":
    st.header("About This Application")
    
    st.markdown("""
    ## Building Energy Consumption Predictor
    
    ### Overview
    This application predicts annual energy consumption for buildings based on architectural, 
    operational, and environmental parameters using machine learning models trained on synthetic data.
    
    ### Research Basis
    Based on the research paper:
    *"Enhancing the Accuracy of Building Performance Simulation through Post-Occupancy Calibration"*
    
    **Key research findings incorporated:**
    - HVAC inefficiencies contribute 38.18% to performance gap
    - Lighting inefficiencies contribute 15.07%
    - Occupancy patterns contribute 14.48%
    - Weather conditions contribute 3.24%
    
    ### Features Used
    1. **Building Characteristics**: Area, floors, age, type
    2. **System Efficiencies**: HVAC, lighting, equipment
    3. **Environmental Factors**: Temperature, humidity, climate zone
    4. **Operational Parameters**: Occupancy hours, usage patterns
    5. **Research-Based Features**: Performance gap factors, efficiency scores
    
    ### Model Architecture
    - **Ensemble Approach**: Combines multiple tree-based models
    - **Best Model**: Random Forest (R²: ~0.92)
    - **Features**: 10+ engineered features
    - **Dataset**: 5,000 synthetic building samples
    
    ### How to Use
    1. Navigate to **Predict Energy** tab
    2. Enter building parameters (basic or advanced)
    3. Click **Predict Energy Consumption**
    4. View results, breakdown, and recommendations
    
    ### Technical Details
    - **Framework**: Streamlit
    - **ML Libraries**: Scikit-learn, XGBoost, LightGBM
    - **Visualization**: Plotly
    - **Deployment**: Local/Cloud ready
    
    ### Limitations
    - Predictions based on synthetic data
    - Regional variations may affect accuracy
    - Actual measurements recommended for critical decisions
    
    ### Future Enhancements
    - Real building data integration
    - Real-time sensor data integration
    - Advanced ML models (deep learning)
    - Carbon footprint optimization
    """)

elif app_mode == "📁 Upload Data":
    st.header("Upload Building Data")
    
    st.info("""
    Upload your own building data (CSV format) to:
    1. Compare with predictions
    2. Enhance model training
    3. Generate custom analyses
    """)
    
    uploaded_file = st.file_uploader(
        "Choose a CSV file",
        type=['csv'],
        help="Upload building data with features similar to the training dataset"
    )
    
    if uploaded_file is not None:
        try:
            df_uploaded = pd.read_csv(uploaded_file)
            
            st.success(f"✅ File uploaded successfully: {len(df_uploaded)} rows, {len(df_uploaded.columns)} columns")
            
            # Display preview
            st.subheader("Data Preview")
            st.dataframe(df_uploaded.head(), use_container_width=True)
            
            # Check for required columns
            required_cols = ['total_floor_area', 'energy_consumption_kwh']
            missing_cols = [col for col in required_cols if col not in df_uploaded.columns]
            
            if missing_cols:
                st.warning(f"⚠️ Missing columns: {', '.join(missing_cols)}")
            else:
                # Basic statistics
                st.subheader("Data Statistics")
                
                col1, col2, col3 = st.columns(3)
                with col1:
                    st.metric(
                        "Average Energy",
                        f"{df_uploaded['energy_consumption_kwh'].mean():,.0f} kWh"
                    )
                with col2:
                    st.metric(
                        "Average Area",
                        f"{df_uploaded['total_floor_area'].mean():,.0f} m²"
                    )
                with col3:
                    st.metric(
                        "Energy Intensity",
                        f"{(df_uploaded['energy_consumption_kwh']/df_uploaded['total_floor_area']).mean():.1f} kWh/m²"
                    )
                
                # Comparison with model predictions
                st.subheader("Comparison with Model Predictions")
                
                artifacts = load_model()
                if artifacts and 'features' in artifacts:
                    features = artifacts['features']
                    
                    # Check which features are available
                    available_features = [f for f in features if f in df_uploaded.columns]
                    
                    if len(available_features) > 0:
                        X_uploaded = df_uploaded[available_features]
                        
                        # Scale and predict
                        scaler = artifacts['scaler']
                        model = artifacts['model']
                        
                        X_scaled = scaler.transform(X_uploaded)
                        predictions = model.predict(X_scaled)
                        
                        # Create comparison
                        comparison_df = pd.DataFrame({
                            'Actual': df_uploaded['energy_consumption_kwh'],
                            'Predicted': predictions,
                            'Difference': df_uploaded['energy_consumption_kwh'] - predictions,
                            'Error %': ((df_uploaded['energy_consumption_kwh'] - predictions) / df_uploaded['energy_consumption_kwh']) * 100
                        })
                        
                        st.dataframe(comparison_df.head(10).style.format({
                            'Actual': '{:,.0f}',
                            'Predicted': '{:,.0f}',
                            'Difference': '{:,.0f}',
                            'Error %': '{:.1f}%'
                        }), use_container_width=True)
                        
                        # Calculate overall metrics
                        mae = np.mean(np.abs(comparison_df['Difference']))
                        mape = np.mean(np.abs(comparison_df['Error %']))
                        
                        col1, col2 = st.columns(2)
                        with col1:
                            st.metric("Mean Absolute Error", f"{mae:,.0f} kWh")
                        with col2:
                            st.metric("Mean Absolute % Error", f"{mape:.1f}%")
        
        except Exception as e:
            st.error(f"Error processing file: {str(e)}")

# Footer
st.sidebar.markdown("---")
st.sidebar.info(
    """
    **Version**: 1.0.0  
    **Last Updated**: December 2025  
    **Environment**: building_energy  
    
    **Note**: For research and educational purposes.  
    Always validate predictions with actual measurements.
    """
)

# Add refresh button
if st.sidebar.button("🔄 Refresh Model"):
    st.cache_resource.clear()
    st.cache_data.clear()
    st.success("Cache cleared! Reloading models...")
    st.rerun()

2025-12-03 13:58:18.355 
  command:

    streamlit run /opt/anaconda3/envs/building_energy/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-12-03 13:58:18.356 No runtime found, using MemoryCacheStorageManager
